## Custom Tensor Factorisation

In previous posts, we have looked at the standard tensor decomposition and also at how to do so using Autograd/Tensorflow. In this post, I'll mention a few custom/non-standard tensor decomposition schemes that may come in handy.

Instead of Khatri product of decomposed matrices to produce the original matrix, we will look at a few dot prodct based combinations.

We will break a matrix E of shape M, N, O using the following four cases:

1. H (M, a), A (N, b), T (O, a, b)
2. H (M, a), A (N, a, b), T (O, b)
3. H (M, a, b), A (N, a), T (O, b)
4. H (M, a), N (N, a), T (O, a)

In [1]:
import autograd.numpy as np
import pandas as pd

In [2]:
M, N, O = 4, 3, 6
E_np = np.random.randn(M, N, O)

In [3]:
E_np = np.abs(E_np)


In [4]:
E_np = np.divide(E_np, E_np.max())


In [5]:
E_np

array([[[ 0.0605506 ,  0.03599664,  0.25087234,  0.33163366,  0.42881236,
          0.34442001],
        [ 0.14693202,  0.12447638,  0.1725975 ,  0.38583806,  0.62645085,
          0.80823158],
        [ 1.        ,  0.17360677,  0.06555334,  0.62725427,  0.12069365,
          0.1828657 ]],

       [[ 0.48998751,  0.38255239,  0.24461023,  0.52672268,  0.37219343,
          0.08538137],
        [ 0.02704806,  0.72177226,  0.22683561,  0.19101462,  0.1277134 ,
          0.60084777],
        [ 0.24456668,  0.49043866,  0.19626124,  0.10446358,  0.21828062,
          0.14397225]],

       [[ 0.24419427,  0.77580798,  0.00324624,  0.28413238,  0.44064043,
          0.79667402],
        [ 0.77413919,  0.71381069,  0.39812893,  0.52891816,  0.66570485,
          0.6308962 ],
        [ 0.61504237,  0.11248786,  0.17295706,  0.50186568,  0.18489032,
          0.37835997]],

       [[ 0.35379831,  0.17485743,  0.23209066,  0.70069727,  0.10180182,
          0.1951613 ],
        [ 0.09460633,  0

### Masking 20% of the entries

In [6]:
E_np_masked = E_np.copy()
for i in range(M):
    for j in range(N):
        for k in range(O):
            if np.random.random()>0.8:
                E_np_masked[i, j, k] = np.NAN

In [7]:
E_np_masked

array([[[        nan,  0.03599664,  0.25087234,  0.33163366,  0.42881236,
          0.34442001],
        [ 0.14693202,  0.12447638,         nan,  0.38583806,  0.62645085,
                 nan],
        [ 1.        ,  0.17360677,  0.06555334,  0.62725427,  0.12069365,
          0.1828657 ]],

       [[ 0.48998751,  0.38255239,         nan,  0.52672268,  0.37219343,
          0.08538137],
        [ 0.02704806,  0.72177226,  0.22683561,         nan,  0.1277134 ,
          0.60084777],
        [ 0.24456668,  0.49043866,  0.19626124,  0.10446358,  0.21828062,
          0.14397225]],

       [[        nan,  0.77580798,  0.00324624,  0.28413238,  0.44064043,
          0.79667402],
        [ 0.77413919,  0.71381069,         nan,  0.52891816,  0.66570485,
          0.6308962 ],
        [ 0.61504237,  0.11248786,  0.17295706,  0.50186568,  0.18489032,
                 nan]],

       [[ 0.35379831,         nan,  0.23209066,  0.70069727,  0.10180182,
                 nan],
        [ 0.09460633,   

In [8]:
a = 2
b = 3

### Case 1 H (M, a), A (N, b), T (O, a, b)

In [9]:
H =  np.random.rand(M, a)
A = np.random.rand(N, b)
T =  np.random.rand(O, a, b)

In [10]:
HA = np.einsum('Ma, Nb -> MNab', H, A)


In [11]:
HAT = np.einsum('MNab, Oab -> MNO', HA, T)
HAT.shape

(4, 3, 6)

### Case 2 H (M, a), A (N, a, b), T (O, b)

In [12]:
H =  np.random.rand(M, a)
A = np.random.rand(N, a, b)
T =  np.random.rand(O, b)

In [13]:
HA = np.einsum('Ma, Nab -> MNb', H, A)
HAT = np.einsum('MNb, Ob ->MNO', HA, T)

We can see that all we need to do is to create the EINSUM strings for the different cases

In [14]:
cases = {
    1: {'HA': 'Ma, Nb -> MNab', 'HAT': 'MNab, Oab -> MNO'},
    2: {'HA': 'Ma, Nab -> MNb', 'HAT': 'MNb, Ob -> MNO'},
    3: {'HA': 'Mab, Na -> MNb', 'HAT': 'MNb, Ob -> MNO'},
    4: {'HA': 'Ma, Na -> MNa', 'HAT': 'MNa, Oa -> MNO'}
}

In [15]:
def multiply_case(H, A, T, case):
    HA = np.einsum(cases[case]['HA'], H, A)
    HAT = np.einsum(cases[case]['HAT'], HA, T)
    return HAT

In [16]:
def cost(H, A, T, E_np_masked, case):
    HAT = multiply_case(H, A, T, case)
    mask = ~np.isnan(E_np_masked)
    error = (HAT-E_np_masked)[mask].flatten()
    return np.sqrt((error**2).mean())


In [17]:
from autograd import multigrad

In [18]:
mg = multigrad(cost, argnums=[0, 1, 2])


In [19]:
lr = 0.1
H =  np.random.rand(M, a)
A = np.random.rand(N, a, b)
T =  np.random.rand(O, b)
case=2

In [20]:
params = {}
params['M'], params['N'], params['O'] = E_np_masked.shape
params['a'] = 2
params['b'] = 3

In [21]:
def learn_HAT(case, E_np_masked, num_iter, lr):
    H_dim_chars = list(cases[case]['HA'].split(",")[0].strip())
    H_dim = tuple(params[x] for x in H_dim_chars)
    A_dim_chars = list(cases[case]['HA'].split(",")[1].split("-")[0].strip())
    A_dim = tuple(params[x] for x in A_dim_chars)
    T_dim_chars = list(cases[case]['HAT'].split(",")[1].split("-")[0].strip())
    T_dim = tuple(params[x] for x in T_dim_chars)
    H =  np.random.rand(*H_dim)
    A = np.random.rand(*A_dim)
    T =  np.random.rand(*T_dim)
    
    # GD procedure
    for i in range(num_iter):
        del_h, del_a, del_t = mg(H, A, T, E_np_masked, case)
        H-=lr*del_h
        A-=lr*del_a
        T-=lr*del_t
        # Projection to non-negative space
        H[H<0] = 0
        A[A<0] = 0
        T[T<0] = 0
        if i%500==0:
            print(cost(H, A, T, E_np_masked, case))
    return H, A, T

In [22]:
learnt_factors = {}
for case_num in range(1, 5):
    print case_num
    print "*"*30
    learnt_factors[case_num] = learn_HAT(case_num, E_np_masked, 2000, 0.1)

1
******************************
0.796066769834
0.174785029807
0.150046840099
0.138501472717
2
******************************
0.592664946527
0.168544673995
0.152961301292
0.150055287512
3
******************************
0.491833921984
0.166327739064
0.142030349507
0.135573587985
4
******************************
0.358511728923
0.181591797797
0.179021490862
0.173692323398


In [30]:
H, A, T = learnt_factors[case]
multiply_case(H, A, T, case)

array([[[ 0.48882969,  0.18708795,  0.25397125,  0.5347884 ,  0.25369064,
          0.19726996],
        [ 0.20039975,  0.23810218,  0.68736954,  0.51351976,  0.39110993,
          0.23696736],
        [ 0.70095996,  0.16928001,  0.20180109,  0.71857591,  0.26201775,
          0.19470855]],

       [[ 0.47607553,  0.40750625,  0.02628398,  0.23101719,  0.25393685,
          0.36988745],
        [ 0.35253324,  0.40849102,  0.29493296,  0.29107321,  0.33595789,
          0.37285137],
        [ 0.31397732,  0.1601071 ,  0.08750033,  0.2674595 ,  0.15029872,
          0.15680797]],

       [[ 0.88203539,  0.77670193,  0.02739895,  0.40008823,  0.4711357 ,
          0.70242502],
        [ 0.66830687,  0.77383461,  0.5103475 ,  0.519137  ,  0.61810599,
          0.70447707],
        [ 0.54619101,  0.29617564,  0.15160879,  0.45387182,  0.26835684,
          0.28735945]],

       [[ 0.40002148,  0.10185109,  0.25811443,  0.50355387,  0.20603974,
          0.12099583],
        [ 0.12819782,  0

SyntaxError: invalid syntax (<ipython-input-27-b29c862b6e6b>, line 1)